***Reference***

https://towardsdatascience.com/prototyping-an-anomaly-detection-system-for-videos-step-by-step-using-lstm-convolutional-4e06b7dcdd29
https://github.com/hashemsellat/Video-Anomaly-Detection/blob/master/lstmautoencoder.ipynb

In [2]:
import warnings
warnings.filterwarnings('ignore')

# Enforce CPU
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"    
import tensorflow as tf
print(tf.test.is_gpu_available())

import numpy as np
import pickle, shutil
import time

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed, Conv2D, ConvLSTM2D, Conv2DTranspose
from tensorflow.keras.layers import BatchNormalization
# from keras_layer_normalization import LayerNormalization
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import TensorBoard, Callback

False


In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Enforce CPU
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"    
import tensorflow as tf
print(tf.test.is_gpu_available())

import numpy as np
import random, pickle

from keras.models import Sequential, load_model
from keras.layers import TimeDistributed, Conv2D, ConvLSTM2D, Conv2DTranspose
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras.callbacks import TensorBoard, Callback
from keras_bert import get_custom_objects

tf.config.threading.set_intra_op_parallelism_threads(1)
tf.config.threading.set_inter_op_parallelism_threads(32)

from PIL import Image

from os import listdir
from os.path import isfile, join, isdir

import seaborn as sns
sns.set_style("white")
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
from utils import get_clips_by_stride, get_train_set, get_single_test
from utils import TRAIN_PATH, TEST_PATH

In [ ]:
def ind2str(ind):
    if ind < 10:
        return '00' + str(ind)
    elif ind < 100:
        return '0' + str(ind)
    else:
        return str(ind)

In [ ]:
# Example Show
dir_ind = 1
img_ind = 10
img_f   = TRAIN_PATH + '/' + 'Train' + ind2str(dir_ind) + '/' + ind2str(img_ind) + '.tif'
img_pil = Image.open(img_f, 'r')
img_pil

***Evaluate model***

**CPU**

In [ ]:
MODEL_PATH = 'out/ucsd.cpu/ucsd.cpu.hdf5'

In [ ]:
model = load_model(MODEL_PATH, custom_objects=get_custom_objects())
model.summary()

In [ ]:
test = get_single_test()
sz   = test.shape[0] - 10
sequences = np.zeros((sz, 10, 256, 256, 1))
# apply the sliding window technique to get the sequences
for i in range(0, sz):
    clip = np.zeros((10, 256, 256, 1))
    for j in range(0, 10):
        clip[j] = test[i + j, :, :, :]
    sequences[i] = clip
sequences.shape    

*Test 32 of UCSDped1. At the beginning of the video, there is a bicycle on the walkway, which explains the low regularity score. After the bicycle left, the regularity score starts to increase. At frame 60, another bicycle enters, the regularity score decreases again and increases right after it left.*

In [ ]:
reconstructed_sequences = model.predict(sequences,batch_size=4)
sequences_reconstruction_cost = np.array([np.linalg.norm(np.subtract(sequences[i],reconstructed_sequences[i])) for i in range(0,sz)])
sa = (sequences_reconstruction_cost - np.min(sequences_reconstruction_cost)) / np.max(sequences_reconstruction_cost)
sr = 1.0 - sa

# plot the regularity scores
plt.plot(sr)
plt.ylabel('regularity score Sr(t)')
plt.xlabel('frame t')
plt.show()

**GPU**

In [ ]:
MODEL_PATH = 'out/ucsd.gpu/ucsd.gpu.hdf5'

In [ ]:
model = load_model(MODEL_PATH, custom_objects=get_custom_objects())
model.summary()

In [ ]:
test = get_single_test()
sz   = test.shape[0] - 10
sequences = np.zeros((sz, 10, 256, 256, 1))
# apply the sliding window technique to get the sequences
for i in range(0, sz):
    clip = np.zeros((10, 256, 256, 1))
    for j in range(0, 10):
        clip[j] = test[i + j, :, :, :]
    sequences[i] = clip
sequences.shape    

In [ ]:
reconstructed_sequences = model.predict(sequences,batch_size=4)
sequences_reconstruction_cost = np.array([np.linalg.norm(np.subtract(sequences[i],reconstructed_sequences[i])) for i in range(0,sz)])
sa = (sequences_reconstruction_cost - np.min(sequences_reconstruction_cost)) / np.max(sequences_reconstruction_cost)
sr = 1.0 - sa

# plot the regularity scores
plt.plot(sr)
plt.ylabel('regularity score Sr(t)')
plt.xlabel('frame t')
plt.show()

## Compare CPU and GPU run time on UCSD

## load recorded time

In [ ]:
import pickle

time_cpu = pickle.load(open('time_cpu.pkl', 'rb'))
time_gpu = pickle.load(open('time_gpu.pkl', 'rb'))

## plot the difference

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def show_diff(cpu_times, gpu_times, check_type):
    cpu_mean = np.mean(cpu_times)
    gpu_mean = np.mean(gpu_times)
    
    cpu_std  = np.std(cpu_times)
    gpu_std  = np.std(gpu_times)
    
    types = ['cpu', 'gpu']
    x_pos = np.arange(len(types))
    
    mean  = [cpu_mean, gpu_mean]
    error = [cpu_std, gpu_std]
    
    # Build the plot
    fig, ax = plt.subplots()
    ax.bar(x_pos, mean, yerr=error, align='center', alpha=0.5, ecolor='black', capsize=10)
    ax.set_ylabel('Average Run Time per ' + check_type)
    ax.set_xticks(x_pos)
    ax.set_xticklabels(types)
    ax.set_title('Run Time Diff between CPU & GPU with UCSD')
    ax.yaxis.grid(True)
    
    # Save the figure and show
    plt.tight_layout()
    plt.savefig('diff_plot_'+check_type+'.png')
    plt.show()

In [ ]:
# time_per_sample
show_diff(time_cpu[0][1:], time_gpu[0][1:], 'Sample')

In [ ]:
# time_per_echo
show_diff(time_cpu[1][1:], time_gpu[1][1:], 'Epoch')

In [ ]:
# time_per_echo
show_diff(time_cpu[2], time_gpu[2], 'Train')